### Write a Program to validate the withdrawal request and display

In [14]:
from abc import ABC, abstractmethod
import uuid

In [38]:
SUCCESS_MSG = "Transaction completed"
FAILED_MSG = "Transaction failed!"

In [39]:
class BankAccount:
    def __init__(self, holder_name: str ,initial_balance: float, minimum_balance: float = 0.0):
        if initial_balance < 0:
            raise ValueError("Initial Balance/Deposit can't be negative")
        
        if not holder_name.strip():
            raise ValueError("Enter a proper holder name")
        
        self.account_no = str(uuid.uuid4().int)[:12]
        self.__balance = initial_balance
        self.minimum_balance = minimum_balance
        self.holder_name = holder_name
        self.cards = []
        self.upi_id = ""

    # This is a READ-ONLY property (Preventing the Direct access of Private variables)
    @property        
    def balance(self):
        return self.__balance
    
    def check_balance(self):
        return f"Your balance is: {self.balance}"
    
    def account_info(self):
        acc = f"Account holder name: {self.holder_name}"
        acc_no = f"Account no: {self.account_no}"
        acc_balance = f"Account balance: {self.balance}"
        acc_min_bal = f"Minimum balance is: {self.minimum_balance}"
        return f"{acc}\n{acc_no}\n{acc_balance}\n{acc_min_bal}"
    
    # These methods are safely accessing the Private Variables
    # To ADD MONEY into MY BANK ACCOUNT
    def deposit(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("Transaction failed, amount can't be less than or equal to zero")
        
        self.__balance += amount
        return SUCCESS_MSG

    # TO TAKE MONEY from MY BANK ACCOUNT
    def withdrawal(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("Amount can't be less than or equal to zero!")
        
        elif amount > self.__balance:
            raise ValueError("Withdrawal amount can't be greater than balance")
        
        elif self.__balance - amount < self.minimum_balance:
            raise ValueError("Minimum balance must be maintained")
        
        self.__balance -= amount
        return FAILED_MSG
    


In [30]:
# Abstract class which will be common for all source of payment gateways
class Payment(ABC):
    @abstractmethod
    def pay(self, account: BankAccount, pin: int, amount: float):
        pass

In [31]:
class Card(ABC):
    @abstractmethod
    def create_card(self, account: BankAccount):
        pass

    @abstractmethod
    def create_pin(self, pin: str, confirm_pin: str) -> int:
        pass

    def generate_card_number(self):
        return str(uuid.uuid4().int)[:16]
    
    @abstractmethod
    def show_card(self):
        pass

In [ ]:
class DebitCard(Card):
    def __init__(self, holder_name: str):
        if not holder_name.strip():
            raise ValueError("Enter a Valid Account holder name!")
        self.holder_name = holder_name
        self.card_no = 0
        self.__pin = 0
    
    # Creates a Pin for the DebitCard
    def create_pin(self, pin: str, confirm_pin: str) -> int:
        if len(pin) != 4:
            raise ValueError("Invalid Pin!")

        if len(confirm_pin) != 4 or confirm_pin != pin:
            raise ValueError("Invalid Pin!")

        return int(pin)
    
    def show_card(self):
        if not self.card_no:
            raise ValueError("No Card is created")
        
        return f"Holder name: {self.holder_name}\nCard Number: {self.card_no}"

    def create_card(self, account: BankAccount, pin: str, confirm_pin: str):
        if not isinstance(account, BankAccount):
            raise ValueError("Invalid Bank Account!")
        
        self.card_no = super().generate_card_number()

        try:
            self.__pin = self.create_pin(pin, confirm_pin)
            self.show_card()
        except ValueError:
            raise

In [41]:
class Upi(Payment):
    def __init__(self, upi_id: str, pin: int) -> None:
        if not upi_id.strip():
            raise ValueError("Invalid UPI Id")
        
        if not isinstance(pin, int):
            raise ValueError("Pin should contain numbers only")
        
        if len(str(pin)) != 4 and len(str(pin)) != 6:
            raise ValueError("PIN must be 4 or 6 digits")

        self.upi_id = upi_id
        self.__pin = pin

    def verify_pin(self, pin):
        return self.__pin == pin

    def pay(self, account: BankAccount, pin: int, amount: float):
        if not isinstance(account, BankAccount):
            raise ValueError("Invalid Bank Account!")
        
        if not self.verify_pin(pin):
            raise ValueError("Invalid PIN")
        
        try:
            account.withdrawal(amount)
            return SUCCESS_MSG
        except ValueError:
            raise

In [34]:
print("""
Enter:
    1 -> To Deposit amount
    2 -> To Withdraw amount
    3 -> To Create a Debit Card
    4 -> Withdraw using UPI
    5 -> To set a Minimum Balance
    6 -> To Check Balance
    7 -> To View Account Info
    8 -> To exit""")


Enter:
    1 -> To Deposit amount
    2 -> To Withdraw amount
    3 -> To Create a Debit Card
    4 -> Withdraw using UPI
    5 -> To set a Minimum Balance
    6 -> To Check Balance
    7 -> To View Account Info
    8 -> To exit


In [ ]:
initial_balance = float(input("Enter a Initial Balance: "))
holder_name = input("Enter a Account holder name: ")
try:
    print(f"Account holder name: {holder_name}\nInitial Balance set: {initial_balance}\n")
    account = BankAccount(holder_name, initial_balance)

    while True: 
        choice = int(input("\nEnter your choice: "))
        print(f"Entered Choice is: {choice}")

        if choice == 1:
            amount = float(input("Enter the deposit amount: "))
            try:
                response = account.deposit(amount)
                print(response)
            except ValueError as e:
                print(f"Error: {e}")
                
        elif choice == 6:
            response = account.check_balance()
            print(response)
        elif choice == 7:
            response = account.account_info()
            print(response)
        elif choice == 8:
            print("Bye!!")
            break
        else:
            print("Enter a valid input")

except ValueError as e:
    print(f"Error: {e}")

Account holder name: Sayan
Initial Balance set: 4500.0

Entered Choice is: 1

Transaction Successful!
Entered Choice is: 6

Your balance is: 5000.0
Entered Choice is: 7

Account holder name: Sayan
Account no: 178786572436
Account balance: 5000.0
Minimum balance is: 0.0
Entered Choice is: 1

Transaction Successful!
Entered Choice is: 6

Your balance is: 55000.0
Entered Choice is: 7

Account holder name: Sayan
Account no: 178786572436
Account balance: 55000.0
Minimum balance is: 0.0
Entered Choice is: 8

Bye!!
